<a href="https://colab.research.google.com/github/ahmedoumar/MovieRecommendationWithTf/blob/main/Recommenders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q tensorflow-recommenders

In [ ]:
from typing import Dict, Text

import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

In [ ]:
# Ratings data:
ratings = tfds.load('movielens/100k-ratings', split='train')
# Features of all the available movies:

movies = tfds.load('movielens/100k-movies', split="train")


ratings = ratings.map(lambda x: {
    'movie_title': x['movie_title'],
    'user_id': x['user_id']
})

movies = movies.map(lambda x: x['movie_title'])

In [ ]:
user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x: x['user_id']))

movie_titles_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
movie_titles_vocabulary.adapt(movies)

## Define a Model

In [ ]:
class MovieLensModel(tfrs.Model):
    """ We derive from a custom base class to help reduce boilerplat.
    under the hood these are still plain keras models"""

    def __init__(
        self,
        user_model: tf.keras.Model,
        movie_model: tf.keras.Model,
        task: tfrs.tasks.Retrieval):

        super().__init__()

        # Set up user and movie representations:
        self.user_model = user_model
        self.movie_model = movie_model

        # Set up a retrieval task:
        self.task = task

    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
        # Define how the loss is computed:

        user_embeddings = self.user_model(features['user_id'])
        movie_embeddings = self.movie_model(features['movie_title'])

        return self.task(user_embeddings, movie_embeddings)


In [ ]:
# Define user and movie models:
user_model = tf.keras.Sequential([
                                  user_ids_vocabulary,
                                  tf.keras.layers.Embedding(user_ids_vocabulary.vocab_size(), 64)
])

movie_model = tf.keras.Sequential([
                                   movie_titles_vocabulary,
                                   tf.keras.layers.Embedding(movie_titles_vocabulary.vocab_size(), 64)
])

# Define your objectives:
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(movies.batch(128).map(movie_model)))

In [ ]:
# Create a retrieval model:
model = MovieLensModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(.5))

# Train for 3 epochs:
hist = model.fit(ratings.batch(4096), epochs=3)

# use brute force search to set up retrieval using the trained representations:

index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    movies.batch(100).map(lambda title: (title, model.movie_model(title)))
)

# Get some recommendations :
_, titles = index(np.array(['42']))

print(f'Top 3 recommendations for user 42: {titles[0, :3]}')

Epoch 1/3
25/25 [==============================] - 17s 561ms/step - factorized_top_k/top_1_categorical_accuracy: 7.0000e-05 - factorized_top_k/top_5_categorical_accuracy: 0.0017 - factorized_top_k/top_10_categorical_accuracy: 0.0045 - factorized_top_k/top_50_categorical_accuracy: 0.0438 - factorized_top_k/top_100_categorical_accuracy: 0.1002 - loss: 33089.6005 - regularization_loss: 0.0000e+00 - total_loss: 33089.6005
Epoch 2/3
25/25 [==============================] - 14s 572ms/step - factorized_top_k/top_1_categorical_accuracy: 1.4000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0046 - factorized_top_k/top_10_categorical_accuracy: 0.0137 - factorized_top_k/top_50_categorical_accuracy: 0.1047 - factorized_top_k/top_100_categorical_accuracy: 0.2100 - loss: 31018.0282 - regularization_loss: 0.0000e+00 - total_loss: 31018.0282
Epoch 3/3
25/25 [==============================] - 14s 570ms/step - factorized_top_k/top_1_categorical_accuracy: 3.5000e-04 - factorized_top_k/top_5_categor